In [22]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [23]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

import numpy as np
import pyphi
import visualize_pyphi
from pyphi.models.subsystem import FlatCauseEffectStructure
import visualize_pyphi.utils as utils
from tqdm.auto import tqdm

In [24]:
# Chapter 3 network

net_name = "ch3"
n=6

# system parameters
strong = 1
medium = 0.2
weak = 0.04
none = 0

# input output
ff = 0.05
fb = 0.005

# naka rushdon params
th = 1 / 4
exp = 5

weights = np.array(
    [
        [strong, none, weak, medium, 0, 0],
        [medium, weak, none, strong, fb, 0],
        [weak, strong, medium, none, 0, 0],
        [none, medium, strong, weak, 0, ff],
        [0, ff, 0, 0, 0, 0],
        [0, 0, 0, fb, 0, 1],
    ]
)
weights[:4,:4] /= weights[:4,:4].sum(1)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = ["s"] * n

network = visualize_pyphi.network_generator.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=False,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [4]:
state = (0, 1, 0, 0, 1, 0)
subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3)) ################
ces = pyphi.compute.ces(subsystem)
sepces = FlatCauseEffectStructure(ces)

In [75]:
relations_compressed = utils.loadpkl('ch3_compressed_allrels.pkl')
relations = [pyphi.relations.Relation.from_indirect_json(subsystem,sepces,cr) for cr in relations_compressed]

In [82]:
k=5

In [84]:
bjces = visualize_pyphi.compute.filter_ces(subsystem,sepces,None,parallel=True,max_relations_k=k,n_jobs=200,relations=[r for r in relations if len(r.relata)<=k])

  0%|          | 0/432 [00:00<?, ?it/s]

[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
[Parallel(n_jobs=200)]: Done 120 out of 432 | elapsed:   16.3s remaining:   42.3s
[Parallel(n_jobs=200)]: Done 294 out of 432 | elapsed:   24.0s remaining:   11.2s
[Parallel(n_jobs=200)]: Done 381 out of 432 | elapsed:   27.9s remaining:    3.7s


Traceback (most recent call last):
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/multiprocessing/queues.py", line 99, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt

Traceback (most recent call last):
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/multiprocessing/queues.py", line 99, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt

Traceback (most recent call last):
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 407, in _process

KeyboardInterrupt: 

line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/multiprocessing/queues.py", line 99, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt

Traceback (most recent call last):
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/multiprocessing/queues.py", line 99, in get
    if not self._rlock.acquire(block, timeout):
KeyboardInterrupt

Traceback (most recent call last):
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py", line 407, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "/home/mgrasso/miniconda3/envs/4.0_3.7/lib/python3.7/multiprocessing/queues.py

In [ ]:
pyphi_ces = pyphi.big_phi.sia(
    subsystem,
    ces,
    [r for r in relations if len(r.relata)<=k],
    parallel=True,
    check_trivial_reducibility=True,
)

In [ ]:
bjphi = bjces[0]['big phi']
pyphiphi = pyphi_ces.phi
print(f'same phis: {bjphi==pyphiphi} {bjphi} {pyphiphi}')

decimal=11
rounded_bjphi = np.round(bjces[0]['big phi'],decimal)
rounded_pyphiphi = np.round(pyphi_ces.phi,decimal)
print(f'same phis (decimal={decimal}): {rounded_bjphi==rounded_pyphiphi} {rounded_bjphi} {rounded_pyphiphi}')

print(f"same ces: {pyphi.big_phi.LinkedCes(bjces[0]['ces'],subsystem)==pyphi_ces.ces}")
print(f"same rels (k<={k}): {set(bjces[0]['relations'])==set(pyphi_ces.relations)} {len(bjces[0]['relations'])} {len(pyphi_ces.relations)}")

In [44]:
# bjphi ces
utils.sepces2df(bjces[0]['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.469923
1,A,EFFECT,A,0,0.613822
2,C,CAUSE,D,0,0.469923
3,C,EFFECT,B,0,0.673202
4,D,CAUSE,B,0,0.469923
5,D,EFFECT,C,0,0.613822
6,AB,CAUSE,ABC,011,0.020929
7,AB,EFFECT,D,0,0.152428
8,AC,CAUSE,AD,01,0.014019
9,AC,EFFECT,AC,00,0.017256


In [45]:
# Pyphi ces
utils.sepces2df(FlatCauseEffectStructure(pyphi_ces.ces),subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.469923
1,A,EFFECT,A,0,0.613822
2,C,CAUSE,D,0,0.469923
3,C,EFFECT,B,0,0.673202
4,D,CAUSE,B,0,0.469923
5,D,EFFECT,C,0,0.613822
6,AB,CAUSE,ABC,011,0.020929
7,AB,EFFECT,D,0,0.152428
8,AC,CAUSE,AD,01,0.014019
9,AC,EFFECT,AC,00,0.017256
